In [19]:
import os
import sys

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append(os.path.join('..', '..'))

import analysis.plotting as pl
import modules.linear_algebra as lin

In [40]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [20]:
"""Script to calculate gait metrics from Kinect data."""

import glob
import os

import numpy as np
from numpy.linalg import norm
import pandas as pd
from sklearn.cluster import MeanShift

import analysis.stats as st
import modules.gait_metrics as gm
import modules.numpy_funcs as nf

load_dir = os.path.join('..', '..', 'data', 'kinect', 'best_pos')
save_dir = os.path.join('..', '..', 'data', 'results')

save_name = 'kinect_gait_metrics.csv'

# All files with .pkl extension
file_paths = sorted(glob.glob(os.path.join(load_dir, '*.pkl')))
save_path = os.path.join(save_dir, save_name)

df_metrics = pd.read_csv(save_path, index_col=0)

for file_path in file_paths[:1]:

    df_head_feet = pd.read_pickle(file_path)
    frames = df_head_feet.index

    # Convert all points to floats to allow for future calculations (e.g. np.isnan)
    df_head_feet = df_head_feet.applymap(lambda point: point.astype(np.float))

    # Remove outliers
    dist_to_foot_l = (df_head_feet.HEAD - df_head_feet.L_FOOT).apply(norm)
    dist_to_foot_r = (df_head_feet.HEAD - df_head_feet.R_FOOT).apply(norm)

    to_filter = st.relative_error(
        dist_to_foot_l, dist_to_foot_r, absolute=True)
    filtered = st.mad_outliers(to_filter, 2)

    good_frames = np.unique(frames[~np.isnan(filtered)])
    keep_frame = np.in1d(frames, good_frames)

    df_head_feet = df_head_feet[keep_frame]
    frames = df_head_feet.index

    # Cluster frames with mean shift to locate the walking passes
    mean_shift = MeanShift(bandwidth=60).fit(nf.to_column(frames))
    labels = mean_shift.labels_
    
    # Sort labels so that the frames are in temporal order
    labels = nf.map_to_whole(labels)

    # DataFrames for each walking pass in a trial
    pass_dfs = nf.group_by_label(df_head_feet, labels)

In [21]:
df_pass = next(pass_dfs)

In [22]:
head_points = np.stack(df_pass.HEAD)
foot_points_1 = np.stack(df_pass.L_FOOT)
foot_points_2 = np.stack(df_pass.R_FOOT)

In [23]:
mean_foot_points = (foot_points_1 + foot_points_2) / 2

In [24]:
normal = lin.unit(np.median(head_points - mean_foot_points, axis=0))

# Line of best fit for head positions
line_point, direction_pass = lin.best_fit_line(head_points)

In [26]:
df_projected = df_head_feet.applymap(lambda x: lin.project_point_plane(x, line_point, normal))

In [32]:
foot_points = np.vstack([foot_points_1, foot_points_2])

In [42]:
fig, ax = plt.subplots()
ax = Axes3D(fig)

pl.scatter3(ax, foot_points, color='k', depthshade=False)

# Calculate axis limits
x_min, x_max = ax.get_xlim3d()
x_range = np.arange(x_min, x_max)
y_min, y_max = ax.get_ylim3d()
y_range = np.arange(y_min, y_max)

pl.plot_plane(ax, point, normal, x_range=x_range, y_range=y_range, alpha=0.2)

AttributeError: module 'analysis.plotting' has no attribute 'plot_plane'